In [1]:
import numpy as np
import pandas as pd
from tpot import TPOTRegressor
import importlib

In [2]:
# Read in the data, paste in the exported pipeline (under directory `tpot_results`):
# train_data = pd.read_csv('processed_data/train_' + split + '_' + imp + '_' + day + '_1h.csv')
train_dat_name = 'METAB_training_preprocessed'
test_dat_name = 'METAB_test_preprocessed'
data_dir = 'qsar/'
label = 'Act'
random_state = 1618
train_data = pd.read_csv(data_dir + train_dat_name + '.csv', index_col = 'MOLECULE')
test_data = pd.read_csv(data_dir + test_dat_name + '.csv', index_col = 'MOLECULE')

testing_features = test_data.drop([label], axis=1)
training_features = train_data.drop([label], axis=1)
training_target = train_data[label]
testing_target = test_data[label]

# Run the TPOT recommended pipeline
pipe_file = 'pipelines.' + train_dat_name + '_' + str(random_state)

In [3]:
pipe = importlib.import_module(pipe_file, package=None)

tpot_results = pipe.opt_pipe(training_features, testing_features)
exported_pipeline = tpot_results['pipe']
training_features = tpot_results['train_feat']
testing_features = tpot_results['test_feat']

tpot_obj= TPOTRegressor()

if hasattr(exported_pipeline, 'steps'): # if it is a pipeline
    tpot_obj._set_param_recursive(exported_pipeline.steps, 'random_state', random_state)
elif hasattr(exported_pipeline, 'random_state'): # if this is a classifier
    exported_pipeline.set_params(**random_state)

# Fit the exported pipeline to the training:
model = exported_pipeline.fit(training_features, training_target)

/Users/ttle/anaconda3/envs/tpotDataSel/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [4]:
pred = exported_pipeline.predict(testing_features)

In [5]:
np.corrcoef(pred, testing_target)[0, 1]

0.796967223019781